## Histone deacetylase 1 - part 1 (dataset preparation)

### Import libraries

In [1]:
import requests
import json
import pandas as pd

from wizepair2.chem import strip_salts

In [2]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


### Define POST request and download 

In [3]:
url = "https://www.ebi.ac.uk/chembl/elk/es/chembl_activity/_search"
headers = {'Content-Type': 'application/json'}
with open('hdac1_inhibitors.chembl') as f:
    payload = f.read()
response = requests.post(url, headers=headers, data=payload)
response

<Response [200]>

### Load results into a pandas data frame

In [4]:
df = json.loads(response.text)['hits']['hits']
df = pd.DataFrame(df)._source
df = pd.json_normalize(df)
df.sample(3)

,activity_properties,standard_units,standard_type,standard_relation,data_validity_comment,activity_comment,target_pref_name,bao_label,pchembl_value,assay_chembl_id,...,_metadata.assay_data.cell_chembl_id,_metadata.assay_data.assay_organism,_metadata.assay_data.tissue_chembl_id,_metadata.assay_data.assay_parameters,_metadata.assay_data.assay_tissue,_metadata.source.src_description,ligand_efficiency.lle,ligand_efficiency.sei,ligand_efficiency.bei,ligand_efficiency.le
108,[],mL.min-1.kg-1,CL,=,None,None,Rattus norvegicus,organism-based format,None,CHEMBL927953,...,None,Rattus norvegicus,None,[],None,Scientific Literature,NaN,NaN,NaN,NaN
26,[],nM,Solubility,>,None,None,No relevant target,small-molecule physicochemical format,None,CHEMBL927951,...,None,None,None,[],None,Scientific Literature,NaN,NaN,NaN,NaN
96,[],microM.hr.kg/umol,AUC,=,None,None,Canis familiaris,organism-based format,None,CHEMBL922576,...,None,Canis lupus familiaris,None,[],None,Scientific Literature,NaN,NaN,NaN,NaN


### Summarise the most common molecules.

In [5]:
df[['molecule_chembl_id', '_metadata.parent_molecule_data.compound_key']].value_counts()[0:10]

molecule_chembl_id  _metadata.parent_molecule_data.compound_key
CHEMBL403813        13b                                            23
CHEMBL256985        14b                                            11
CHEMBL258177        15b                                            11
CHEMBL402208        13a                                            11
CHEMBL403812        13d                                            11
CHEMBL257176        3                                               9
CHEMBL258176        15a                                             7
CHEMBL402542        14c                                             7
CHEMBL255244        14a                                             6
CHEMBL255245        13c                                             6
dtype: int64

### Summarise the most common targets.

In [6]:
df[['assay_chembl_id', 'target_pref_name', 'bao_label', 'standard_type', 'standard_units']].value_counts()[0:10]

assay_chembl_id  target_pref_name       bao_label                              standard_type  standard_units
CHEMBL927950     HERG                   single protein format                  IC50           nM                13
CHEMBL927948     Histone deacetylase 1  cell-based format                      IC50           nM                13
CHEMBL927949     HCT-116                cell-based format                      IC50           nM                13
CHEMBL927951     No relevant target     small-molecule physicochemical format  Solubility     nM                12
CHEMBL927953     Rattus norvegicus      organism-based format                  CL             mL.min-1.kg-1      4
CHEMBL927956     Rattus norvegicus      organism-based format                  AUC            ng.hr.mL-1         4
CHEMBL927955     Rattus norvegicus      organism-based format                  Vdss           L.kg-1             4
CHEMBL927954     Rattus norvegicus      organism-based format                  T1/2   

### Consider logD measurements to be pchembl values

In [7]:
df['pchembl_value'] = df.apply(lambda x: x['standard_value'] if x['standard_type'] == 'LogD' else x['pchembl_value'], axis=1)

### Strip salts from the molecular structures.

In [8]:
df['stripped_smiles'] = df.canonical_smiles.parallel_apply(strip_salts)
df.stripped_smiles.unique().size

13

### Split training from test set and save

In [9]:
training = ~df['_metadata.parent_molecule_data.compound_key'].isin(['13c', '14d'])

In [10]:
df[training].to_csv('hdac1_inhibitors_stripped.csv', index=False)
df[~training].to_csv('hdac1_inhibitors_stripped_holdout.csv', index=False)